#### Q1. 고객 속성(id, 성별, 연령, 거주지역) 데이터와 거래 데이터를 통합한 데이터 프레임 df를 생성하세요.

고객별 속성 데이터 

- id: 1번 ~ 100번 일련번호
- gender: 0, 1 정수, 난수 생성
- age: 10 ~ 80 사이 정수 난수를 생성하고 범주화하여 age_cat 파생
- region: 1 ~ 5 사이 난수 생성

거래 데이터 

- id: 1번 ~ 100번 일련번호 (관측치 1000개)
- product: 고급제품(hpd), 일반제품(ldp)로 구분
- price: 100 ~ 200 사이 정수 난수 생성
- qty: 1 ~ 2 사이 정수 난수 생성
- day: 요일
- date: 2020-01-01 ~ 2021-12-31 사이의 날짜 1000개 생성
- amount: price * qty 로 산출 

#### Q2. df를 수정하여 5가지 이상의 인사이트를 포함한 데이터 셋으로 변환하세요. 

#### Q3. df를 탐색적 분석을 통하여 인사이트를 도출하세요.

## Q. 타이타닉 생존자 예측모델 개발을 위한 Titanic 분석용 데이터셋

#### Titanic data 전처리
- 분석 데이터 : titanic3.csv
- 재사용 가능한 전처리 사용자 함수 작성 하여 전처리
    - Null 값 처리 : Age는 평균나이, 나머지 칼럼은 'N'값으로 변경
    - 불필요한 속성 칼럼 삭제
    - 문자열 칼럼 레이블 인코딩
- 통계적, 시각적 탐색을 통한 다양한 인사이트 도출
- 탐색적 분석을 통한 feature engineering, 파생변수 

#### 컬럼 정보

- survived : 생존여부(1: 생존, 0 : 사망)
- pclass : 승선권 클래스(1 : 1st, 2 : 2nd ,3 : 3rd)
- name : 승객 이름
- sex : 승객 성별
- age : 승객 나이
- sibsp : 동반한 형제자매, 배우자 수
- parch : 동반한 부모, 자식 수
- ticket : 티켓의 고유 넘버
- fare 티켓의 요금
- cabin : 객실 번호
- embarked : 승선한 항구명(C : Cherbourg, Q : Queenstown, S : Southampton)
- boat
- body
- home.dest

In [9]:
import pandas as pd
df = pd.read_csv('dataset/titanic3.csv')
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.00,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.00,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [10]:
df.drop(['name','ticket','boat','body','home.dest','cabin'],axis=1,inplace=True)

In [21]:
df = df.dropna(subset=['age'],how='any',axis=0) # age의 결측값 드랍
df.isnull().sum()

pclass      0
survived    0
sex         0
age         0
sibsp       0
parch       0
fare        1
embarked    2
dtype: int64

In [23]:
df['fare'].fillna(df['fare'].mean(),inplace=True)
df.isnull().sum()

pclass      0
survived    0
sex         0
age         0
sibsp       0
parch       0
fare        0
embarked    2
dtype: int64

In [25]:
most_em = df['embarked'].value_counts(dropna=True).idxmax()
df['embarked'].fillna(most_em,inplace=True)
df.isnull().sum()

pclass      0
survived    0
sex         0
age         0
sibsp       0
parch       0
fare        0
embarked    0
dtype: int64

In [31]:
df.head()

,pclass,survived,sex,age,sibsp,parch,fare,embarked
0,1,1,female,29.00,0,0,211.3375,S
1,1,1,male,0.92,1,2,151.5500,S
2,1,0,female,2.00,1,2,151.5500,S
3,1,0,male,30.00,1,2,151.5500,S
4,1,0,female,25.00,1,2,151.5500,S


In [35]:
df.head()
def age_cat(a):
    if a <= 20 : cat = 1
    elif a <= 30 : cat = 2
    elif a <= 40 : cat = 3
    elif a <= 50 : cat = 4
    elif a <= 60 : cat = 5
    else : cat = 6
    return cat

def fare_cat(a):
    if a == 0 : result = 0
    elif a <= 10 : result = 1
    elif a <= 30 : result = 2
    elif a <= 60 : result = 3
    elif a <= 90 : result = 4
    else : result = 5
    return result

df['age_cat'] = df.age.apply(age_cat)
df['fare_cat'] = df.fare.apply(fare_cat)

df2 = df.drop(['fare','age','embarked'],axis=1)
df2

,pclass,survived,sex,sibsp,parch,age_cat,fare_cat
0,1,1,female,0,0,2,5
1,1,1,male,1,2,1,5
2,1,0,female,1,2,1,5
3,1,0,male,1,2,2,5
4,1,0,female,1,2,2,5
...,...,...,...,...,...,...,...
1301,3,0,male,0,0,4,1
1304,3,0,female,1,0,1,2
1306,3,0,male,0,0,2,1
1307,3,0,male,0,0,2,1


In [36]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df2['sex'] = le.fit_transform(df2['sex'])

df2

,pclass,survived,sex,sibsp,parch,age_cat,fare_cat
0,1,1,0,0,0,2,5
1,1,1,1,1,2,1,5
2,1,0,0,1,2,1,5
3,1,0,1,1,2,2,5
4,1,0,0,1,2,2,5
...,...,...,...,...,...,...,...
1301,3,0,1,0,0,4,1
1304,3,0,0,1,0,1,2
1306,3,0,1,0,0,2,1
1307,3,0,1,0,0,2,1


In [38]:
y = df2['pclass']
x = df2.drop('pclass',axis=1)

In [39]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# 독립변수, 종속변수 분리
y_t_df = y # 종속변수
X_t_df = x # 독립변수

# 독립변수 정규화
# X_t_df = preprocessing.StandardScaler().fit(X_t_df).transform(X_t_df)

# 학습용 데이터와 평가용 데이터를 8:2 혹은 7:3으로 분리
X_train, X_test, y_train, y_test = train_test_split(X_t_df, y_t_df, test_size = 0.2,
                                                   random_state = 11)

print(X_train.shape)
print(X_test.shape)


(836, 6)
(210, 6)


In [56]:
# 모델 학습 및 평가
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
accuracy_rf = accuracy_score(y_test, rf_pred).round(2)

lr_model = LogisticRegression()
lr_model.fit(X_train,y_train)
lr_pred = lr_model.predict(X_test)
accuracy_lr = accuracy_score(y_test,lr_pred).round(2)

print('rf 정확도:{}, lr 정확도:{}'.format(accuracy_rf,accuracy_lr))

rf 정확도:0.82, lr 정확도:0.85


## 모델 성능 개선 및 평가

### 데이터셋 개선을 위한 검토사항 예시
* 변수 = 'age_cat','male','female','fare_cat','family',{town_C','town_Q','town_S'}
* age의 Null 값을 평균값으로 대체하면 전체적인 데이터의 왜곡이 심함을 확인
* pclass는 fare_cat이랑 같이 모델에 넣을 경우 정확도가 떨어지고(0.82) 각각 넣었을 때는 fare_cat을 넣었을 때의 정확도가 더 높음(0.82, 3% 차이). 이상치에 가까울 정도로 요금이 높은 사람의 경우 사망률이 3클래스 승객과 비슷한 수치를 보이는 점이 pclass 변수에서는 반영이 되지 않았던 것이 원인으로 추측
* sex, embarked 변수로 집어넣은 것보다 원핫 인코더(dummies)로 처리해서 넣는 것이 정확도를 대략 5% 정도 높여주며 디시전 트리의 분기를 더 쉽게하는 효과 확인
* parch와 sibsp를 각각 변수에 적용하면  의미 있는 양상이 보이지 않고 정확도를 떨어뜨리지만 두 변수를 합쳐서 family라는 파생변수를 생성하면 생존율이 높은 여성 승객일지라도 가족 구성원 수가 많으면 생존율이 낮아지는 것을 확인

### 전처리 내역 예시
- age null 처리방법 변경 : 평균값 대체 > 삭제
- pclass와 fare_cat중 분석변수 선택
- embarked 원핫인코딩